## Perform a non-linear fit on a Zn spectrum
This notebook will fit a Gaussian peak shape model to the three distinct peaks in a Zn spectrum.
  * Fit and remove the background
  * Isolate the data in each fit region
  * Build a model for each fit region
  * Fit all three regions simultaneously using a non-linear fit algorithm

In [ ]:
using Gadfly
using Revise
using NeXLSpectrum

### Load and Visualize the Spectrum Data

In [ ]:
spec=loadspectrum(joinpath(@__DIR__,"K309","Zn std.msa"))
set_default_plot_size(8inch,3inch)
plot(spec, klms=[n"Zn", n"C", n"O"], xmax=12.0e3)

### Fit a continuum model
  * Build a `detector` to describe the basic detector properties and a `detectorresponce` matrix to describe the detector efficiency.
  * Fit a continuum model (returned as a `Spectrum`) to the spectral data.
  * Plot the results

In [ ]:
# Build a suitable detector model
det = matching(spec, 132.0, 100)
# Build a detector response model matrix
resp = detectorresponse(det,SDDEfficiency(AP33Model()))
# Fit a contiunuum model to the spectrum data
brem = fittedcontinuum(spec,det,resp)
# Display the result
plot(spec, brem, autoklms=true, yscale=0.02)

### Compute the background corrected spectrum
  * The background corrected data should contain only the characteristic intensity

In [ ]:
bkgcorr = spec-brem
plot(bkgcorr, autoklms=true, yscale=0.04)

### Determine fit regions
  * The fit regions are regions of contiguous overlapping peaks
  * In Zn, there are 3 (L-family, Kα and Kβ) 
  * The third argument determines the width (which we want to include peak and adjacent background)
  * I'll use the `labeledextents(...)` function which returns a `Vector{Tuple{Vector{CharXRay}, UnitRange{Int64}}}` containing the lists of characteristic X-rays and the ranges of channels in which they are found. 

In [ ]:
lex=NeXLSpectrum.labeledextents(n"Zn", det, 1.0e-9)

Plot each of the ranges of channels that we will fit.

In [ ]:
hstack( ( plot(x=lex[i][2], y=bkgcorr[lex[i][2]], Geom.line) for i in eachindex(lex))...)

In [ ]:
using LsqFit
using BenchmarkTools

The function `total_ff2(lex, unk)` computes the input arguments for `LsqFit.curve_fit(...)` based on the X-rays and ranges-of-channels in `lex` and the background-corrected unknown spectrum in `unk`.  The input arguments for `LsqFit.curve_fit(...)` are a function, the x-data and the y-data to be fit.

In [ ]:
function total_ff2(lex, bkgcorr)
    # Build functions for each region
    l = sum(le->length(le[2]), lex)
    x, y, i = zeros(l), zeros(l), 1
    for (cxrs, roc) in lex
        for ch in roc
            x[i] = ch
            y[i] = bkgcorr[ch]
            i+=1
        end
    end 
    data = Dict(cxr=>(energy(cxr), weight(cxr)) for cxr in union(map(le->Set(le[1]), lex)...))
    # This is the fit function
    f(chs, args) = map(chs) do ch 
        eCh, fwhm = args[1] + ch*args[2], args[3]
        res = 0.0
        for (i, (cxrs, roc)) in enumerate(lex)
            if ch in roc
                res += args[i+3] * sum(cxrs) do cxr
                    e0, wgt = data[cxr]
                    g = sqrt(2.45 * (e0 - enx"Mn K-L3") + fwhm^2) / (2.0*sqrt(2.0*log(2.0)))
                    wgt*exp(-0.5*(((e0-eCh)/g)^2))/(g*sqrt(2π))
                end
            end
        end
        res
    end
    # Return the inputs to the LsqFit.curve_fit function
    return (f, x, y)
end

Let's construct some initial arguments from which to start the fit.

In [ ]:
args = [ energy(1,spec), channelwidth(1,spec), 132.0, 2e6, 7e5, 8.0e5]
(f2, x, y) = total_ff2(lex, bkgcorr)
plot(x=x, y=f2(x, args), Geom.line)

In [ ]:
res=curve_fit(f2, x, y, args)
plot(
    layer(x=x, y=y, Geom.line, Theme(default_color="red")),
    layer(x=x, y=f2(x, res.param), Geom.point, Theme(default_color="blue")),
    layer(x=x, y=100.0*res.resid, Geom.line, Theme(default_color="green")),
)

In [ ]:
@show res.param;

Compare to [ -481.8, 5.0049, 126.7, ...] in DTSA-II

Now let's try to refit using a function that can vary the intensity of each `CharXRay` independently. We start with the result of the previous fit.

In [ ]:
function total_ff3(lex, bkgcorr)
    # Build functions for each region
    l = sum(le->length(le[2]), lex)
    x, y, i = zeros(l), zeros(l), 1
    for (cxrs, roc) in lex
        for ch in roc
            x[i] = ch
            y[i] = bkgcorr[ch]
            i+=1
        end
    end 
    data = [ (cxr, energy(cxr), weight(cxr)) for cxr in Iterators.flatten(map(le->le[1], lex)) ]
    # This is the fit function
    f(chs, args) = map(chs) do ch 
        eCh, fwhm = args[1] + ch*args[2], args[3]
        off = 0
        return sum(lex) do (cxrs, roc)
            if ch in roc
                res=sum(cxrs) do cxr
                    sc = args[(off+=1)+3]
                    cx, e0, wgt = data[off]
                    @assert cx == cxr "$cx != $cxr"
                    g = sqrt(2.45 * (e0 - enx"Mn K-L3") + fwhm^2) / (2.0*sqrt(2.0*log(2.0)))
                    sc*wgt*exp(-0.5*(((e0-eCh)/g)^2))/(g*sqrt(2π))
                end
            else
                off+=length(cxrs)
                res=0.0
            end
            res
        end
    end
    # Return the inputs to the LsqFit.curve_fit function
    return (f, x, y)
end

Check the function `total_ff3(...)` produces the same results as `total_ff2(...)` for the same input arguments.

In [ ]:
(f3, x, y) = total_ff3(lex, bkgcorr)
args3 = [ res.param[1:3]..., collect(Iterators.flatten((fill(res.param[3+i],length(lex[i][1])) for i in eachindex(lex))))...]
plot(
    layer(x=x, y=f2(x, res.param), Geom.line, Theme(default_color="blue")),
    layer(x=x, y=f3(x, args3), Geom.point, Theme(default_color="red"))
)


Run the curve fit on the higher dimensionality model function.

Plot the results.   Note the residuals are generally multiplied by a factor of 100.0 to make them visible.

In [ ]:
res3 = curve_fit(f3, x, y, args3)
plot(
    layer(x=x, y=y, Geom.line, Theme(default_color="red")),
    layer(x=x, y=f3(x, res3.param), Geom.line, Theme(default_color="blue")),
    layer(x=x, y=100.0*res3.resid, Geom.line, Theme(default_color="green")),
)

The residual is really tiny (as you might expect with so many free parameters.)

But sadly, the weights are junk. We can have negative weights.

In [ ]:
@show res3.param;

Let's try to add some constraints on the arguments.

In [ ]:
lb = [ args3[1], args3[2], args3[3], (a*0.9 for a in args3[4:end])...]
ub = [ args3[1], args3[2], args3[3], (a*1.1 for a in args3[4:end])...]
res3 = curve_fit(f3, x, y, args3, lower=lb, upper=ub)
plot(
    layer(x=x, y=y, Geom.line, Theme(default_color="red")),
    layer(x=x, y=f3(x, res3.param), Geom.line, Theme(default_color="blue")),
    layer(x=x, y=res3.resid, Geom.line, Theme(default_color="green")),
)

It takes forever and doesn't actually produce very good results.

In [ ]:
plot(x=x, y=res3.resid, Geom.line, Theme(default_color="green"))

In [ ]:
@show res3.param;

Let's be a little bit more clever.  We'll constrain the line intensity to be positive through the use of by making the function argument `ap = log(a)` and inverting the argument back using `exp(ap)` in the function.  Since the range of the function `exp(...)` is positive definite, we can't get negative intensities.

In [ ]:
function total_ff4(lex, bkgcorr)
    # Build functions for each region
    l = sum(le->length(le[2]), lex)
    x, y, i = zeros(l), zeros(l), 1
    for (cxrs, roc) in lex
        for ch in roc
            x[i] = ch
            y[i] = bkgcorr[ch]
            i+=1
        end
    end 
    # data is visible within f
    data = [ (cxr, energy(cxr), weight(cxr)) for cxr in Iterators.flatten(map(le->le[1], lex)) ]
    # This is the fit function
    f(chs, args) = map(chs) do ch 
        eCh, fwhm = args[1] + ch*args[2], args[3]
        off = 0
        return sum(lex) do (cxrs, roc)
            if ch in roc
                res=sum(cxrs) do cxr
                    cx, e0, wgt = data[off+=1]
                    @assert cx == cxr "$cx != $cxr"
                    @assert off+3 >= 4
                    sc = exp(args[off+3])
                    g = sqrt(2.45 * (e0 - enx"Mn K-L3") + fwhm^2) / (2.0*sqrt(2.0*log(2.0)))
                    sc*wgt*exp(-0.5*(((e0-eCh)/g)^2))/(g*sqrt(2π))
                end
            else
                off+=length(cxrs)
                res=0.0
            end
            res
        end
    end
    # Return the inputs to the LsqFit.curve_fit function
    return (f, x, y)
end

In [ ]:
(f4, x, y) = total_ff4(lex, bkgcorr)
args4 = [ res.param[1:3]..., collect(Iterators.flatten(fill(log(res.param[3+i]),length(lex[i][1])) for i in eachindex(lex)))...]
plot(
    layer(x=x, y=f2(x, res.param), Geom.line, Theme(default_color="blue")),
    layer(x=x, y=f4(x, args4), Geom.point, Theme(default_color="red"))
)

In [ ]:
res4 = curve_fit(f4, x, y, args4)
@show res4.param;

In [ ]:
plot(
    layer(x=x, y=y, Geom.line, Theme(default_color="red")),
    layer(x=x, y=f4(x, res4.param), Geom.line, Theme(default_color="blue")),
    layer(x=x, y=100.0*res4.resid, Geom.line, Theme(default_color="green")),
)

In [ ]:
data = [ (cxr, energy(cxr), weight(cxr)) for cxr in Iterators.flatten(map(le->le[1], lex)) ]
collect(zip(data, exp.(res4.param[4:end]), exp.(res4.param[4:end]) ./ exp.(args4[4:end])))

This is better but there remains a problem.  Lines we know are intense have been diminished and lines we know are weaker have been intensified.  We know this is a problem.  So simply constraining the line intensities to be positive isn't enough.  We also need to constrain them to be within a certain range of their tabulated values - intense lines can't become weak and weak lines can't become intense.

Compare to [ -481.8, 5.0049, 126.7, ...] in DTSA-II

In [ ]:
@show res4.param[1:3];

In [ ]:
function range_constrain(center, width) 
    return ( 
        res -> tan((res-center)/(width*(2.0/π))), # From constrained
        arg -> center + width*(2.0/π)*atan(arg) # To constrained
    )
end

In [ ]:
(f, fi) = range_constrain(100.0,10.0)
g(x) = f(x), fi(f(x))
g(90.0001), g(100.0), g(101.0), g(109.9999)

In [ ]:
function db(w) 
    if w>0.9
        return 0.1
    elseif w>0.5
        return 0.5
    elseif w>0.1
        return 0.8
    elseif w>0.01
        return 0.9
    else
        return 0.99
    end
end
lb = [ args4[1]-10.0, args4[2]*0.999, 120.0, ( a*(1.0 - db(d[3])) for (d, a) in zip(data, args4[4:end]))... ]
ub = [ args4[1]+10.0, args4[2]/0.999, 150.0, (a*(1.0 + db(d[3])) for (d, a) in zip(data, args4[4:end]))... ]
#lb, ub
res4 = curve_fit(f4, x, y, args4, lower=lb, upper=ub)
@show res4.param;

In [ ]:
collect(zip(data, exp.(res4.param[4:end]), exp.(res4.param[4:end]) ./ exp.(args4[4:end])))

In [ ]:
@show res4.param[1:3];

In [ ]:
plot(
    layer(x=x, y=res4.resid, Geom.line, Theme(default_color="green")),
)

We are going to have to be much more sophisticated to get the fit we want.

This block implements a handful of different useful constraints.  Essentially constraints are functions that map (-∞,∞) to a constrained range such as (0,∞), (min, max), (center-width, center+width) etc.  The optimization still occurs on the unbounded interval (-∞,∞) but the result is constrained to within the range.

In [ ]:
abstract type ConstrainedParameter end
initial(uc::ConstrainedParameter) = uc.val

struct Unconstrained <: ConstrainedParameter 
    val::Float64
end

forward(uc::Unconstrained, v::Float64) = v
back(uc::Unconstrained, v::Float64) = v

struct PositiveConstraint <: ConstrainedParameter
    val::Float64
end

forward(uc::PositiveConstraint, v::Float64) = log(v)
back(uc::PositiveConstraint, v::Float64) = exp(v)

struct NegativeConstraint <: ConstrainedParameter 
    val::Float64
end

forward(uc::NegativeConstraint, v::Float64) = log(-v)
back(uc::NegativeConstraint, v::Float64) = -exp(v)

struct RangeConstraint <: ConstrainedParameter 
    val::Float64
    width::Float64
end

forward(uc::RangeConstraint, v::Float64) = tan((v-uc.val)/(uc.width*(2.0/π)))
back(uc::RangeConstraint, v::Float64) = uc.val + uc.width*(2.0/π)*atan(v)

struct MinMaxConstraint <: ConstrainedParameter 
    val::Float64
    min::Float64
    max::Float64
end
back(uc::MinMaxConstraint, v::Float64) = tan((v-0.5*(uc.min+uc.max))/((uc.max-uc.min)*(2.0/π)))
forward(uc::MinMaxConstraint, v::Float64) = 0.5*(uc.min+uc.max) + (uc.max-uc.min)*(2.0/π)*atan(v)

In [ ]:
rc = MinMaxConstraint(100.0, 92.0, 110.0)
g(rc, v) = forward(rc, v), back(rc, forward(rc, v))
g(rc, 110.0-0.001)

In [ ]:
rc = PositiveConstraint(100.0)
g(rc, v) = forward(rc, v), back(rc, forward(rc, v))
g(rc, 10000.001)

In [ ]:
rc = RangeConstraint(100.0, 10.0)
g(rc, v) = forward(rc, v), back(rc, forward(rc, v))
g(rc, 110.0-0.001)

In [ ]:
rc = Unconstrained(100.0)
g(rc, v) = forward(rc, v), back(rc, forward(rc, v))
g(rc, 110.0-0.001)

Here we define how the fitting function arguments are constrained.  The energy calibration and resolution calibration are handled independently.  The intensity calibration is handled such that intense lines remain intense.  This ensures that the curve_fit function doesn't play off the energy scale against the intensities such that intensity gets transferred from intense lines to weak lines.

In [ ]:
function iconst(w, i)
    if w>0.5
        return RangeConstraint(i, 0.1*i)
    elseif w>0.1
        return RangeConstraint(i, 0.2*i)
    elseif w>0.01
        return RangeConstraint(i, 0.8*i)
    else
        return PositiveConstraint(i)
    end
end 

args5 = [ 
    RangeConstraint(args3[1], 10.0),
    RangeConstraint(args3[2], 0.01),
    RangeConstraint(args3[3], 10.0),
    ( iconst(d[3], ii) for (d, ii) in zip(data, args3[4:end]))...
];
@show args5;

Now construct a function that takes as input the constrained parameters and, through the magic of closures, applies the constraints to keep the arguments to the function `f` within a constrained range.

In [ ]:
function constrained_curve_fit(f, x, y, cp0::AbstractVector{<:ConstrainedParameter}; kwargs...)
    g(x, args) = f(x, [ back(cp, arg) for (cp, arg) in zip(cp0, args) ])
    p0 = [ forward(cp, initial(cp)) for cp in cp0 ]
    res = curve_fit(g, x, y, p0; kwargs...)
    rp = [ back(cv, arg) for (cv, arg) in zip(cp0, res.param) ]
    return LsqFit.LsqFitResult(rp, res.resid, res.jacobian, res.converged, res.wt)
end

In [ ]:
res5=constrained_curve_fit(f3, x, y, args5)
@show res5.param;

In [ ]:
plot(
    layer(x=x, y=100.0*res5.resid, Geom.line, Theme(default_color="green")),
    layer(x=x, y=y, Geom.line, Theme(default_color="red")),
    layer(x=x, y=f3(x, res5.param), Geom.point)
)

This looks a whole lot better.  The residual is relatively small and relatively structureless.  The weight is in the lines we expect it to be.

In [ ]:
collect(zip(data, res5.param[4:end], res5.param[4:end] ./ args3[4:end]))